In [23]:
import pandas as pd
import numpy as np
import z3
# from tqdm.notebook import tqdm
# from tqdm.autonotebook import tqdm
from tqdm import tqdm
# blah blah your code errored
tqdm._instances.clear()


In [24]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60
NUM_FLOW = 50
DATA_NAME = "0"
TOPO_NAME = "2"

task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:50]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

## 1. Model

Network model:

$\left\langle\left[v_{a}, v_{b}\right] . s,\left[v_{a}, v_{b}\right] . d,\left[v_{a}, v_{b}\right] . m t,\left[v_{a}, v_{b}\right] . c\right\rangle$

In [25]:
s = z3.Optimize()
s.set("timeout", 5 * 60*60*1000)

In [26]:
## Set macrotick as 100 ms

U = 3
E = max(network['t_proc'])
MSS = 150 
LCM = np.lcm.reduce(task['period'])

In [27]:
net_var = {}
for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['msd'] = row['t_proc']
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

In [28]:
task_var = {}

In [29]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [30]:
## Assume task is strictly periodic
for i, row in task.iterrows():
    task.loc[i,'route'] = str(next(bfs_paths(net, int(row['src']), int(eval(row['dst'])[0]))))
    task_var.setdefault(i, {})
    route = eval(task.loc[i, 'route'])
    for j in range(U):
        task_var[i].setdefault(j, {})
        task_var[i][j]['w'] = z3.Int('w_' +  str(i) + '_' + str(j))
        task_var[i][j]['s'] =  z3.Int('s_' +  str(i) + '_' + str(j))
        task_var[i][j]['u'] =  z3.Int('u_' +  str(i) + '_' + str(j))
    task_var[i]['p'] = int(row['period'])
    task_var[i]['d'] = int(row['deadline'])
    task_var[i]['s'] = int(row['size'])
    task_var[i]['pi'] = list({x: int(eval(str(route))[i+1]) for i, x in enumerate(eval(str(route))[:-1])}.items())
    

## 2. Constraints

Range Constraints:

\begin{aligned}
&\forall f_i \in F, \forall j \in\left[0, \frac{H}{p_i}\right), \forall g \in[0, U) \\
&0 \leq s_{i, j, g} \leq M S S, u_{i, j, g} \in\{0,1\} \\
&j \times p_i \leq w_{i, j, g} \leq j \times p_i+d_i \\
&w_{i, j, g} \leq w_{i, j, g+1}, u_{i, j, g} \geq u_{i, j, g+1} .
\end{aligned}

In [31]:
for i in task_var:
    for g in range(U):
        s.add(
            task_var[i][g]['s'] >= 0,
            task_var[i][g]['s'] <= MSS,
            task_var[i][g]['w'] >= 0,
            task_var[i][g]['w'] < task_var[i]['p'],
            task_var[i][g]['w'] < task_var[i][g + 1]['w'] if g + 1 < U else True,
            task_var[i][g]['u'] >= 0,
            task_var[i][g]['u'] <= 1,
            task_var[i][g]['u'] >= task_var[i][g + 1]['u'] if g + 1 < U else True
        )

Size Constraint

\begin{gathered}
\forall f_i \in F, \forall j \in\left[0, \frac{H}{p_i}\right), \sum_{\forall g \in[0, U)} s_{i, j, g}=\mathfrak{s}_i \\
\forall g \in[0, U),\left(\left(s_{i, j, g}>0\right) \wedge\left(u_{i, j, g}=1\right)\right) \vee \\
\left(\left(s_{i, j, g}=0\right) \wedge\left(u_{i, j, g}=0\right)\right)
\end{gathered}

In [32]:
for i in task_var:
    s.add(
        z3.Sum(
            [task_var[i][g]['s'] for g in range(U)]
        ) == task_var[i]['s']
    )
    for g in range(U):
        s.add(
            z3.Or(
                z3.And(task_var[i][g]['s'] > 0, task_var[i][g]['u'] == 1),
                z3.And(task_var[i][g]['s'] == 0, task_var[i][g]['u'] == 0)
            )
        )

Real-Time Constraint
\begin{aligned}
&\forall f_i \in F, \forall j \in\left[0, \frac{H}{p_i}\right), \forall g \in[0, U) \\
&\neg u_{i, j, g} \vee\left(w_{i, j, g}+\frac{s_{i, j, g}+e}{v} \times r_i \leq j \times p_i+d_i\right)
\end{aligned}


In [33]:
for i in task_var:
    for g in range(U):
        s.add(
            z3.Or(
                task_var[i][g]['u'] == 0,
                (task_var[i][g]['s'] * 8 + net_var[str(task_var[i]['pi'][0])]['msd'] + sync_error)\
                     * len(task_var[i]['pi']) < task_var[i]['d']
            )
        )

No-Conflict Constraint:

\begin{aligned}
&\forall l_{a, b} \in L, \forall f_i, f_x \in F, \forall j \in\left[0, \frac{H}{p_i}\right) \\
&\forall y \in\left[0, \frac{H}{p_x}\right), \forall g, z \in[0, U) \\
&((i=x) \wedge(j=y) \wedge(g=z)) \vee \\
&\neg u_{i, j, g} \vee \neg u_{x, y, z} \vee\left(l_{a, b} \notin \Pi_i\right) \vee\left(l_{a, b} \notin \Pi_x\right) \vee \\
&\left(A_{x, y, z}\left(l_{a, b}\right)+\frac{s_{x, y, z}+e}{v}<A_{i, j, g}\left(l_{a, b}\right)\right) \vee \\
&\left(A_{i, j, g}\left(l_{a, b}\right)+\frac{s_{i, j, g}+e}{v}<A_{x, y, z}\left(l_{a, b}\right)\right)
\end{aligned}

In [34]:
## Construct A_x,y,z

A = {}
for i in task_var:
    A.setdefault(i, {})
    for g in range(U):
        A[i].setdefault(g, {})
        route = task_var[i]['pi']
        for hop, link in enumerate(route):
            if hop == 0:
                A[i][g][link] = task_var[i][g]['w']
            else:
                A[i][g][link] = A[i][g][route[hop - 1]] + task_var[i][g]['s'] * 8 + net_var[str(route[hop])]['msd'] + sync_error

In [35]:
for link in tqdm(net_var):
    link = eval(link)
    for i, j in [(i,j) for i in task_var for j in task_var 
                if link in task_var[i]['pi'] and link in task_var[j]['pi']]:
        lcm = np.lcm(task_var[i]['p'], task_var[j]['p'])
        for a, b in [(a,b) for a in range(0, int(lcm / task_var[i]['p'])) for b in range(0, int(lcm / task_var[j]['p']))]:
            for g, z in [(g,z) for g in range(U) for z in range(U)]:
                if i < j or g != z:
                    s.add(
                        z3.Implies(
                            task_var[i][g]['u'] + task_var[j][z]['u'] == 2,
                            z3.Or(
                                A[i][g][link] + (task_var[i][g]['s']) * 8 + a * task_var[i]['p'] < A[j][z][link] + b * task_var[j]['p'],
                                A[j][z][link] + (task_var[j][z]['s']) * 8 + b * task_var[j]['p'] < A[i][g][link] + a * task_var[i]['p'],
                            )
                        )
                    )

100%|██████████| 40/40 [00:04<00:00,  8.72it/s]


## Objective 

In [36]:
# s.minimize(
#     z3.Sum(
#         [task_var[i][g]['u'] for i,g in [(i, g) for i in task_var for g in range(U)]]
#     )
# )

In [37]:
res = s.check()

In [ ]:
result = s.model()

## Output schedule

In [ ]:
## GCL
GCL = []
for i in task_var:
    for hop, e in enumerate(task_var[i]['pi']):
        for u in [u for u in range(U) if result[task_var[i][u]['u']] == 1]:
            start = result.eval(A[i][u][e]).as_long()
            size = result[task_var[i][u]['s']].as_long() * 8
            end = start + size
            queue = 0
            t = task_var[i]['p']
            for k in range(int(LCM / t)):
                GCL.append(
                    [e, queue, (start + k * t) * macrotick, (end + k * t) * macrotick, LCM * macrotick]
                )

In [ ]:
## Offset
OFFSET = []
for i in task_var:
    for u in [u for u in range(U) if result[task_var[i][u]['u']] == 1]:
        offset = result.eval(A[i][u][task_var[i]['pi'][0]]).as_long()
        OFFSET.append(
            [i, u, (task_var[i]['p'] - offset) * macrotick]
        )    

In [ ]:
QUEUE = []
for i in task_var:
    for e in task_var[i]['pi']:
        for u in [u for u in range(U) if result[task_var[i][u]['u']] == 1]:
            QUEUE.append([i, 0, e, 0])
        # queue[i][e] = result[task_var[i][e]['p']]
# with open('RTNS16-queue-CBS-16.txt', 'w') as f:
#     f.write(str(queue))

In [ ]:
ROUTE = []
for i, row in task.iterrows():
    route = task_var[i]['pi']
    for h, v in enumerate(route):
        ROUTE.append(
            [i, v]
        )

In [ ]:
SIZE = []
for i in task_var:
    for u in [u for u in range(U) if result[task_var[i][u]['u']] == 1]:
        SIZE.append(
            [i, u, result[task_var[i][u]['s']].as_long() * macrotick]
        )

In [ ]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("IEEEJAS2021-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("IEEEJAS2021-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("IEEEJAS2021-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("IEEEJAS2021-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

SIZE = pd.DataFrame(QUEUE)
SIZE.columns = ['id','ins_id','link','queue']
SIZE.to_csv("IEEEJAS2021-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)